In [7]:
import pandas as pd

# df = pd.read_csv('../data/all_train_results_finetunedllama.csv')
# df = pd.read_csv('../data/all_train_pairs_combined_results_finetunedModel.csv')
df = pd.read_csv('../data/all_train_results_18_dec.csv')

In [8]:
str(df)

"                                           question_text  \\\n0          How many actions are depicted in the diagram?   \n1          How many actions are depicted in the diagram?   \n2          How many actions are depicted in the diagram?   \n3          How many actions are depicted in the diagram?   \n4      How many steps are there to the formation of a...   \n...                                                  ...   \n15151  any force that is used to move an object does ...   \n15152                 work is done when force is applied   \n15153             the amount of work done depends on the   \n15154  you do more work lifting an object if the obje...   \n15155  you do more work playing basketball than you d...   \n\n                         question_image  \\\n0      question_images/erosion_6843.png   \n1      question_images/erosion_6843.png   \n2      question_images/erosion_6843.png   \n3      question_images/erosion_6843.png   \n4      question_images/erosion_6843.png   \

In [9]:
df

,question_text,question_image,answer_choices,context_image,context_image_caption,text_context,label,predicted_answer,logits
0,How many actions are depicted in the diagram?,question_images/erosion_6843.png,"a. 6, b. 4, c. 8, d. 7",teaching_images/erosion_6859.png,The diagram represents the coastal Erosion of ...,Most fossils are preserved by one of five proc...,d,d,"[{'letter': 'd', 'logit': 27.125, 'probability..."
1,How many actions are depicted in the diagram?,question_images/erosion_6843.png,"a. 6, b. 4, c. 8, d. 7",teaching_images/erosion_6859.png,The diagram represents the coastal Erosion of ...,Most fossils are preserved by one of five proc...,d,d,"[{'letter': 'd', 'logit': 27.125, 'probability..."
2,How many actions are depicted in the diagram?,question_images/erosion_6843.png,"a. 6, b. 4, c. 8, d. 7",teaching_images/erosion_6859.png,The diagram represents the coastal Erosion of ...,Most fossils are preserved by one of five proc...,d,d,"[{'letter': 'd', 'logit': 27.125, 'probability..."
3,How many actions are depicted in the diagram?,question_images/erosion_6843.png,"a. 6, b. 4, c. 8, d. 7",teaching_images/erosion_6859.png,The diagram represents the coastal Erosion of ...,Most fossils are preserved by one of five proc...,d,d,"[{'letter': 'd', 'logit': 27.125, 'probability..."
4,How many steps are there to the formation of a...,question_images/erosion_6843.png,"a. 6, b. 5, c. 10, d. 7",teaching_images/erosion_6859.png,The diagram represents the coastal Erosion of ...,"Despite these problems, there is a rich fossil...",d,d,"[{'letter': 'd', 'logit': 26.875, 'probability..."
...,...,...,...,...,...,...,...,...,...
15151,any force that is used to move an object does ...,NaN,"a. true, b. false",textbook_images/work_23180.png,FIGURE 1.1,Work is the use of force to move an object. It...,b,b,"[{'letter': 'b', 'logit': 27.25, 'probability'..."
15152,work is done when force is applied,NaN,"a. for a long enough period of time., b. in th...",textbook_images/what_is_force_22255.png,FIGURE 13.2 Forces can vary in both strength a...,Work is the use of force to move an object. It...,c,c,"[{'letter': 'c', 'logit': 27.5, 'probability':..."
15153,the amount of work done depends on the,NaN,"a. amount of force applied., b. distance the o...",textbook_images/work_23180.png,FIGURE 1.1,Work is the use of force to move an object. It...,d,d,"[{'letter': 'd', 'logit': 27.375, 'probability..."
15154,you do more work lifting an object if the obje...,NaN,"a. heavier., b. bigger., c. harder., d. warmer.",textbook_images/work_23181.png,FIGURE 1.2,Work is the use of force to move an object. It...,a,a,"[{'letter': 'a', 'logit': 27.5, 'probability':..."


In [10]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import ast
import json

def evaluate_predictions(df):
    # Convert logits string to actual list/dict
    def get_highest_logit_letter(logits_str):
        try:
            # Convert string representation to list of dicts
            logits_list = ast.literal_eval(logits_str)
            # Find the dict with highest logit value
            max_logit_dict = max(logits_list, key=lambda x: x['logit'])
            return max_logit_dict['letter']
        except:
            return None

    # Get predictions based on highest logits
    df['predicted_label'] = df['logits'].apply(get_highest_logit_letter)
    
    # Remove rows where predictions or labels are None/NaN
    mask = df['predicted_label'].notna() & df['label'].notna()
    y_true = df.loc[mask, 'label']
    y_pred = df.loc[mask, 'predicted_label']
    print(f'GT: {y_true}\n Pred: {y_pred}\n\n')

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

    # Return confusion matrix as DataFrame
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Usage:
# df = pd.read_csv('your_file.csv')
# metrics = evaluate_predictions(df)

In [11]:
metrics = evaluate_predictions(df[:100])

GT: 0     d
1     d
2     d
3     d
4     d
     ..
95    d
96    b
97    b
98    d
99    a
Name: label, Length: 100, dtype: object
 Pred: 0     d
1     d
2     d
3     d
4     d
     ..
95    d
96    b
97    b
98    d
99    a
Name: predicted_label, Length: 100, dtype: object


Accuracy: 0.9400
F1 Score: 0.9408
Precision: 0.9435
Recall: 0.9400


/var/folders/50/ddkl_7r93pv5mq0_h58b4lv80000gn/T/ipykernel_99642/920685615.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicted_label'] = df['logits'].apply(get_highest_logit_letter)


In [6]:
df.iloc[1].logits

"[{'letter': 'd', 'logit': 26.9375, 'probability': 0.7752557396888733}, {'letter': 'c', 'logit': 24.984375, 'probability': 0.10995464026927948}, {'letter': 'a', 'logit': 24.953125, 'probability': 0.10657169669866562}, {'letter': 'b', 'logit': 22.390625, 'probability': 0.008217927068471909}]"